# Running Wflow using the ewatercycle system

### 1. Copy an example case
To run WFlow, we need a complete parameterset. The easiest way to obtain a valid model configuration is by copying it from https://github.com/openstreams/wflow/raw/master/examples/. We can use `ewatercycle.parametersetdb` to easily copy on of these example cases to a folder called `./wflow_example_case`.

In [ ]:
from ewatercycle.parametersetdb import build_from_urls

# Obtain an example case for testing the model
parameterset = build_from_urls(
    config_format='ini', config_url='https://github.com/openstreams/wflow/raw/master/examples/wflow_rhine_sbm/wflow_sbm.ini',
    datafiles_format='svn', datafiles_url='https://github.com/openstreams/wflow/trunk/examples/wflow_rhine_sbm',
)
parameterset.save_datafiles('./wflow_example_case')

### 2. Verify your configuration settings

If you are working on a curated environment like https://lab.ewatercycle.org/, the default configuration should cover 90% of all use cases. However, for the other 10% here is a brief explanation of the relevant configuration settings. 

The ewatercycle configuration file is typcially stored in `~/ewatercycle/ewatercycle.yaml` or `/etc/ewatercycle.yaml` and can be imported with `from ewatercycle import CFG`. Relevant settings for WFLow are:

```Yaml
container_engine: docker # or singularity
singularity_images.wflow: 'ewatercycle-wflow-grpc4bmi.sif'
docker_images.wflow: 'ewatercycle/wflow-grpc4bmi:latest'
scratch_dir: './' # the directory where the model work_dir will be created
```

In order for this to work, you need to make sure that the actual docker or singularity images are available on the system. This can be done with:

- Docker: `docker pull ewatercycle/wflow-grpc4bmi:latest`
- Singularity: `singularity build --disable-cache ewatercycle-wflow-grpc4bmi.sif docker://ewatercycle/wflow-grpc4bmi:latest`

### 3. Setting up the model

In [1]:
from ewatercycle.models import Wflow

In [2]:
model = Wflow()

In [3]:
model.parameters

First run setup with a valid .ini file.


In [4]:
# This should work for a well-defined example case
# cfg_dir, cfg_file = model.setup(cfg_dir='wflow_example_case/', cfg_file='./wflow_example_case/wflow_sbm.ini')

# You can also pass custom forcing data (for now through a simple string):
# cfg_dir, cfg_file = model.setup(cfg_dir='wflow_example_case/', cfg_file='./wflow_example_case/wflow_sbm.ini', forcing_data="path/to/forcing.nc")

# And you can add custom settings to overwrite whatever is in the cfg_file. 
# This is useful, as the WFLOW version we're using is inconsistent with the example config file. We need to add the RiverRunoff variable to the API section in the .ini-file:
cfg_dir, cfg_file = model.setup(cfg_dir='wflow_example_case/', cfg_file='./wflow_example_case/wflow_sbm.ini', API={"RiverRunoff": "2, m^3/s"})

Working directory created: wflow_20210414_160743
Created /home/peter/ewatercycle/ewatercycle/examples/wflow_20210414_160743/wflow_ewatercycle.ini.
Started wflow container with working directory /home/peter/ewatercycle/ewatercycle/examples/wflow_20210414_160743


Note 

In [5]:
cfg_dir, cfg_file

('/home/peter/ewatercycle/ewatercycle/examples/wflow_20210414_160743',
 'wflow_ewatercycle.ini')

The `config_file` that is returned by `setup(...)` can be used to initialize the model:

In [6]:
# Print a list of currently used parameters
model.parameters

inputmapstacks
    Precipitation: /inmaps/P
    EvapoTranspiration: /inmaps/PET
    Temperature: /inmaps/TEMP
    Inflow: /inmaps/IF
run
    starttime: 1995-01-31 00:00:00 GMT
    endtime: 1995-02-28 00:00:00 GMT
    runinfo: None
    timestepsecs: 86400
    reinit: 0
model
    modeltype: sbm
    AnnualDischarge: 2290
    Alpha: 120
    ModelSnow: 0
    ScalarInput: 0
    InterpolationMethod: inv
    WIMaxScale: 0.8
    Tslice: 1
    UpdMaxDist: 300000.0
    origTopogLateral: 1
    configfile: wflow_sbm.ini
    reinit: 0
    fewsrun: 0
    OverWriteInit: 0
    updating: 0
    updateFile: no_set
    lateralmethod: 1
    sCatch: 0
    intbl: intbl
    timestepsecs: 86400
    MaxUpdMult: 1.3
    MinUpdMult: 0.7
    UpFrac: 0.8
    waterdem: 0
    reInfilt: 0
    MassWasting: 0
    wflow_subcatch: staticmaps/wflow_subcatch.map
    wflow_dem: staticmaps/wflow_dem.map
    wflow_ldd: staticmaps/wflow_ldd.map
    wflow_river: staticmaps/wflow_river.map
    wflow_riverlength: staticmaps/wflow_r

### 4. Running the model

In [7]:
model.initialize(cfg_file)

In [8]:
model.get_value("RiverRunoff")

array([-999., -999., -999., ..., -999., -999., -999.])

In [9]:
model.get_value_as_xarray("RiverRunoff")  # this automatically converts the -999 values to NaN

<xarray.DataArray 'RiverRunoff' (latitude: 169, longitude: 187)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * longitude  (longitude) float64 5.227 5.264 5.3 5.337 ... 11.97 12.01 12.05
  * latitude   (latitude) float64 45.89 45.93 45.97 46.0 ... 51.98 52.02 52.05
    time       object 1995-01-30 00:00:00
Attributes:
    units:     m^3/s

In [ ]:
while model.time < model.end_time:
    print(model.time)
    model.update()

791424000.0
791510400.0
791596800.0
791683200.0
791769600.0
791856000.0
791942400.0
792028800.0
792115200.0
792201600.0
792288000.0
792374400.0
792460800.0
792547200.0
792633600.0
792720000.0
792806400.0
792892800.0
792979200.0
793065600.0
793152000.0
793238400.0
793324800.0


In [ ]:
da = model.get_value_as_xarray("RiverRunoff")
da.plot(robust=True, cmap='GnBu')

In [ ]:
model.finalize()

In [ ]:
del(model)